# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
# загружаем все необходимые библиотеки
import pandas as pd
import numpy as np

import nltk
import torch
import transformers
from nltk.corpus import stopwords as nltk_stopwords
from tqdm import notebook
from pymystem3 import Mystem

from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifierCV
)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    f1_score,
    make_scorer
)
import matplotlib.pyplot as plt

from scipy.stats import uniform

## Подготовка

In [2]:
# считываем данные комментариев
try:
    df_comm = pd.read_csv("c:/users/bny7/toxic_comments.csv")
except:
    df_comm = pd.read_csv("/datasets/toxic_comments.csv")

In [3]:
# выведем на экран данные для просмотра
display(df_comm.head())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
# переименуем первый столбец, потому что исходный имеет весьма странное название
df_comm.rename(columns={'Unnamed: 0': 'num'})

,num,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [5]:
# выведем на экран информацию о наборе данных
display(df_comm.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

In [6]:
# подсчитаем длину комментариев в каждой строке
list=[]
for i in df_comm['text']:
    list.append(len(i))

# добавим поле в набор данных
df_comm['len']=list

Модель Bert имеет ограничение на 512 токенов, поэтому максимальная длина должна быть не более 512.     
Подсчитаем количество строк, у которых длина более 512 слов:

In [7]:
# подсчитаем, сколько комментариев имеют длину более 512
display(df_comm.query('len>512')['len'].count())

# создадим отдельный набор с количеством токенов менее 512
df_comm_512 = df_comm.query('len<=512')

32943

Около 20% всех записей. Не мало. Будем использовать только TF-IDF

In [8]:
%%time
# добавим в наш набор столбец с лемматизированным текстом
corpus=[]
m=Mystem()

for i in df_comm['text']:
    new_i = "".join(m.lemmatize(i))
    corpus.append(new_i)

CPU times: user 30.9 s, sys: 7.77 s, total: 38.7 s
Wall time: 1min 29s


In [9]:
df_comm['text_lem'] = corpus
display(df_comm['text_lem'][:5])

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: text_lem, dtype: object

**Вывод:**    
- данные загружены и преобразованы в датафрейм;
- данные лемматизированы и очищены;
- принято решение, что для построения модели мы не будем использовать Bert, а воспользуемся привычными инструментами с использованием TF-IDF.

## Обучение

### Разделим выборку на обучающую, тестовую и валидационную и сформируем TF-IDF

In [10]:
X = df_comm['text_lem']
y = df_comm['toxic']
TEST_SIZE=0.4
VAL_SIZE=0.5
RANDOM_STATE=42

In [11]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE
)

In [12]:
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=VAL_SIZE,
    random_state=RANDOM_STATE
)

In [13]:
# загружаем стоп-слова
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
vectorize = TfidfVectorizer(stop_words=stopwords)

In [15]:
%%time
tf_idf_train = vectorize.fit_transform(X_train)
tf_idf_test = vectorize.transform(X_test)
tf_idf_valid = vectorize.transform(X_valid)

CPU times: user 7.64 s, sys: 0 ns, total: 7.64 s
Wall time: 7.64 s


In [16]:
display(tf_idf_train.shape, tf_idf_test.shape, tf_idf_valid.shape)

(95575, 138806)

(31859, 138806)

(31858, 138806)

Поскольку задача относится к классификации, то модель будем строить с использованием:
- LogisticRegression,
- RidgeClassifierCV,
- RandomForestClassifier.

### Логистическая регрессия

In [17]:
%%time
lr = LogisticRegression(penalty='l1', solver='saga')
lr.fit(tf_idf_train, y_train)

CPU times: user 25.7 s, sys: 0 ns, total: 25.7 s
Wall time: 25.7 s


LogisticRegression(penalty='l1', solver='saga')

In [18]:
%%time
display("F1 на обучающей выборке: ", cross_val_score(lr, tf_idf_train, y_train, scoring='f1', cv=3))

'F1 на обучающей выборке: '

array([0.74432023, 0.75236903, 0.75982063])

CPU times: user 44 s, sys: 0 ns, total: 44 s
Wall time: 44 s


Метрика F1 для LogisticRegression на обучающей выборке составила **0,75**:    
array([0.74432023, 0.75236903, 0.75982063])

In [20]:
# посмотрим, что модель определила как токсичные сообщения
tt_lr=lr.predict(tf_idf_test)-y_test
display(df_comm.iloc[tt_lr[tt_lr!=0].index])

,Unnamed: 0,text,toxic,len,text_lem
67856,67924,and when i done popin 17 1/2 capp in yo black ...,1,150,and when i done popin 17 1/2 capp in yo black ...
155007,155164,"""\n\nThat's great that you observed how it's o...",0,373,"""\n\nThat's great that you observed how it's o..."
151166,151322,"ISIS massacred many civilians, including child...",1,102,"ISIS massacred many civilians, including child..."
155918,156076,", you are a fake and a fraud as a Wikipedia ed...",1,204,", you are a fake and a fraud as a Wikipedia ed..."
51240,51296,An announcement i would like to make: \n\nI am...,1,169,An announcement i would like to make: \n\nI am...
...,...,...,...,...,...
8653,8665,You undid my changes to the A.G.S.B article \n...,1,297,You undid my changes to the A.G.S.B article \n...
113967,114065,Thanks a lot! \n\nThanks for deleting my artic...,1,164,Thanks a lot! \n\nThanks for deleting my artic...
151179,151335,Why can't you find a significant other\noh wai...,1,145,Why can't you find a significant other\noh wai...
64163,64230,"""\n\nROTFL! I've not found that idiot, but I t...",0,308,"""\n\nROTFL! I've not found that idiot, but I t..."


По словам **massacred, fraud** можно поверить, что модель отработала неплохо.     
Общее количество отбракованных сообщений на тестовой выборке составило **1282**.

### RIDGE

In [21]:
model_ridge = RidgeClassifierCV(cv=3,scoring='f1')

In [22]:
%%time
model_ridge.fit(tf_idf_train, y_train)

CPU times: user 1min 22s, sys: 1min 32s, total: 2min 55s
Wall time: 2min 55s


RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ]), cv=3, scoring='f1')

In [23]:
print(model_ridge.score(tf_idf_train, y_train))

0.971561600837039


Метрика f1 для Ridge на обучающей выборке составила **0,97**

In [25]:
y_pred_ridge = model_ridge.predict(tf_idf_test)

In [26]:
# посмотрим, что модель определила как токсичные сообщения
tt=y_pred_ridge-y_test
display(df_comm.iloc[tt[tt!=0].index])

,Unnamed: 0,text,toxic,len,text_lem
4088,4088,"I swear, you're extremely stupid and oblivious...",1,109,"I swear, you're extremely stupid and oblivious..."
62381,62448,"""\n\n I have a better idea. \n\nTell your """"fr...",1,78,"""\n\n I have a better idea. \n\nTell your """"fr..."
151166,151322,"ISIS massacred many civilians, including child...",1,102,"ISIS massacred many civilians, including child..."
155918,156076,", you are a fake and a fraud as a Wikipedia ed...",1,204,", you are a fake and a fraud as a Wikipedia ed..."
51240,51296,An announcement i would like to make: \n\nI am...,1,169,An announcement i would like to make: \n\nI am...
...,...,...,...,...,...
113967,114065,Thanks a lot! \n\nThanks for deleting my artic...,1,164,Thanks a lot! \n\nThanks for deleting my artic...
6904,6912,Hey... Dont add any crap you like... agarwals ...,1,162,Hey... Dont add any crap you like... agarwals ...
151179,151335,Why can't you find a significant other\noh wai...,1,145,Why can't you find a significant other\noh wai...
37583,37629,"""your undo of my edit iis totaly stupid im red...",1,137,"""your undo of my edit iis totaly stupid im red..."


По словам **massacred, fraud** можно поверить, что модель отработала качественно.     
Общее количество отбракованных сообщений на тестовой выборке составило **1550**.

### RandomForestClassifier

In [27]:
rf = RandomForestClassifier(n_jobs=-1,
                            random_state=RANDOM_STATE)

In [28]:
%%time
rf.fit(tf_idf_train, y_train)

CPU times: user 5min 40s, sys: 0 ns, total: 5min 40s
Wall time: 5min 40s


RandomForestClassifier(n_jobs=-1, random_state=42)

In [29]:
%%time
display("F1 на обучающей выборке: ", cross_val_score(rf, tf_idf_train, y_train, scoring='f1', cv=3))

'F1 на обучающей выборке: '

array([0.69394056, 0.70082202, 0.70826178])

CPU times: user 29min 38s, sys: 0 ns, total: 29min 38s
Wall time: 29min 40s


Метрика **F1** RandomForestClassifier на обучающей выборке составила **0,70**:  array([0.69394056, 0.70082202, 0.70826178])

In [31]:
# посмотрим, что модель определила как токсичные сообщения
tt_rf=rf.predict(tf_idf_test)-y_test
display(df_comm.iloc[tt_rf[tt_rf!=0].index])

,Unnamed: 0,text,toxic,len,text_lem
67856,67924,and when i done popin 17 1/2 capp in yo black ...,1,150,and when i done popin 17 1/2 capp in yo black ...
150025,150181,I am not planning any personal attack except o...,1,304,I am not planning any personal attack except o...
151166,151322,"ISIS massacred many civilians, including child...",1,102,"ISIS massacred many civilians, including child..."
159036,159195,http://www.nysun.com/article/23698 - public in...,1,59,http://www.nysun.com/article/23698 - public in...
155918,156076,", you are a fake and a fraud as a Wikipedia ed...",1,204,", you are a fake and a fraud as a Wikipedia ed..."
...,...,...,...,...,...
113967,114065,Thanks a lot! \n\nThanks for deleting my artic...,1,164,Thanks a lot! \n\nThanks for deleting my artic...
6904,6912,Hey... Dont add any crap you like... agarwals ...,1,162,Hey... Dont add any crap you like... agarwals ...
151179,151335,Why can't you find a significant other\noh wai...,1,145,Why can't you find a significant other\noh wai...
79206,79282,"""\nI beg to differ. His edit summary clearly s...",0,99,"""\nI beg to differ. His edit summary clearly s..."


По словам **massacred, fraud** можно поверить, что модель отработала качественно.     
Общее количество отбракованных сообщений на тестовой выборке составило **1509**.

По результатам построения модели на обучающей выборке порядок такой:     
- Метрика f1 для Ridge на обучающей выборке составила **0,97**;
- Метрика F1 для LogisticRegression на обучающей выборке составила **0,75**;
- Метрика F1 RandomForestClassifier на обучающей выборке составила **0,70**.

### Проводим оценку моделей на валидационной выборке

In [32]:
%%time
f1_lr = f1_score(y_valid, lr.predict(tf_idf_valid))

CPU times: user 14.6 ms, sys: 0 ns, total: 14.6 ms
Wall time: 12.9 ms


In [33]:
%%time
f1_ridge = f1_score(y_valid, model_ridge.predict(tf_idf_valid))

CPU times: user 12.7 ms, sys: 0 ns, total: 12.7 ms
Wall time: 12.4 ms


In [34]:
%%time
f1_rf = f1_score(y_valid, rf.predict(tf_idf_valid))

CPU times: user 5.7 s, sys: 0 ns, total: 5.7 s
Wall time: 5.73 s


In [36]:
display("F1 на валидационной выборке моделью ridge: ", f1_ridge)
display("F1 на валидационной выборке логистической моделью: ", f1_lr)
display("F1 на валидационной выборке моделью случайный лес: ", f1_rf)

'F1 на валидационной выборке моделью ridge: '

0.6833855799373042

'F1 на валидационной выборке логистической моделью: '

0.7609081234480313

'F1 на валидационной выборке моделью случайный лес: '

0.6958534233365478

Выбираем модель **LogisticRegression**.

### Оценим выбранную модель LogisticRegression на тестовой выборке

In [37]:
%%time
f1_lr = f1_score(y_test, lr.predict(tf_idf_test))

CPU times: user 15.7 ms, sys: 0 ns, total: 15.7 ms
Wall time: 14.2 ms


In [38]:
display("F1 на тестовой выборке логистической моделью: ", f1_lr)

'F1 на тестовой выборке логистической моделью: '

0.7695073714491191

Метрика **F1** на тестовой выборке составила **0,76**, что удовлетворяет условию задачи.

## Выводы

1. Данные исходных сообщений загружены и сохранены в датафрейм.
2. Проведена подготовка данных, загружены стоп-слова, лемматизирован текст, получен TF-IDF для корпуса текстов, выбраны и обучены 3 модели:
    - LogisticRegression,
    - RidgeClassifierCV,
    - RandomForestClassifier.
3. Построенная модель логистической регрессии (LogisticRegression) показала результаты:
    - на обучающей выборке метрика f1 составила 0,75;
    - количество найденных угроз на тестовой выборке = 1282.
4. Построенная модель классификации RidgeClassifier со встроенной крос-валидацией (RidgeClassifierCV) показала результаты:
    - на обучающей выборке метрика f1 составила 0,97;
    - количество найденных угроз на тестовой выборке = 1522.   
5. Построенная модель классификации Случайный лес (RandomForestClassifier) показала результаты:
    - на обучающей выборке метрика f1 составила 0,70;
    - количество найденных угроз на тестовой выборке = 1509.  
6. Провели тестирование построенных моделей на валидационной выборке:
    - на тестовой выборке метрика f1 LogisticRegression составила **0,76**;
    - на тестовой выборке метрика f1 RandomForestClassifier составила **0,70**;    
    - на тестовой выборке метрика f1 RidgeClassifier составила **0,68**;
8. Отобрали модель LogisticRegression как лучшую.  
7. Проверили выбранную модель на тестовой выборке и получили метрику f1 = 0,76, что удовлетворяет условию задачи.

**Итоги:   лучшей моделью по качеству метрики оказалась LogisticRegression.**